# 一阶段测试
* 主要目的是测试参考代码的可行性和准确性

In [ ]:
import numpy as np
from sympy import *
import re

In [ ]:
def func(x):
    return x**4
x = symbols("x")
print(diff(func(x),x))   # 表示将x = 2，带入导函数4*x**3中

In [ ]:
def WolfePowell(f,d,x,alpha_,rho,sigma):
    maxk = 1000  #迭代上限
    k = 0
    phi0 = fun(x) # fun(x)
    dimensions = dim(f) # f_

    dx = []
    grad = []
    item={}
    for i in range(dimensions):
        item[x_[i]] = x[i]    
    for a in range(dimensions):
        dx.append(diff(f, symbols('x'+str(a),real=True))) #求偏导

        grad.append(dx[a].subs(item)) #求梯度
        
    phi0_ = np.dot(grad,d)
    print(dx)
    print(grad)
    
    a1 = 0
    a2 = alpha_
    alpha = (a1+a2)/2
    phi1 = phi0
    phi1_ = phi0_

    k = 0
    for k in range(maxk):    #限制迭代上限,避免时间太长
        phi = fun(x + alpha * d) #fun
        if phi <= phi1 + rho * alpha * phi1_:
            newx = x + alpha * d
            newdx = []
            newgrad = []
            for a in range(dimensions):
                newdx.append(diff(f, symbols('x'+str(a),real=True)))  # 求偏导
                newitem={}
                for i in range(dimensions):
                    newitem[x_[i]] = newx[i]
                newgrad.append(newdx[a].subs(newitem)) #求梯度

            phi_ = np.dot(newgrad,d)
            
            if phi_ >= sigma*phi0_:
                break
            else:
                alpha_new = alpha + (alpha-a1) *phi_ / (phi1_-phi_)
                a1 = alpha
                alpha = alpha_new
                phi1 = phi
                phi1_ = phi_
        else:
            alpha_new = a1 + 0.5*(a1-alpha)**2*phi1_/((phi1-phi)-(a1-alpha)*phi1_)
            a2 = alpha
            alpha = alpha_new
    k = k + 1
    return alpha

'''利用正则表达式统计目标函数维度'''
def dim(f_):
    dimension_set = []
    dimension_set = re.findall(r'x[0-9]\d*',str(f_))
    dimensions = len(set(dimension_set))
    return dimensions

In [ ]:
'''测试'''
x_ = []
for a in range(10):
    x_.append(symbols('x'+str(a),real=True))  #设置符号变量
   
def fun(x):
    return 100*(x[1]-x[0]**2)**2+(1-x[0])**2  #目标函数
f_ = fun(x_)   #用于W-P

alpha_ = 1  #alpha_max
rho = 0.25  # rho∈（0,1/2） 
sigma = 0.5  # sigma∈（rho，1）
x = np.random.rand(dim(f_))   #随机的初始点
d = np.array([-1,-1])     #初始方向

alpha=WolfePowell(f_,d,x,alpha_,rho,sigma)
print(alpha)


In [ ]:
diff(f_, symbols('x'+str(0),real=True))

# 二阶段测试
* 测试对象为打包好的非精确一维搜索类
* 该编自参考代码 https://blog.csdn.net/weixin_43761124/article/details/107436454

In [ ]:
from search import *
def fun(x):
    return 100*(x[1]-x[0]**2)**2+(1-x[0])**2  #目标函数

searcher = Line_Searcher()
searcher.add_fun(fun)

In [ ]:
from search import _grad
_grad(fun, [1,2])

In [ ]:
x = np.array([1,2])
d = np.array([-1,-1])     #初始方向
searcher.WolfePowell(x,np.array([4,-2]))

In [ ]:
fun([1,2]) - fun([1,2]+0.087*np.array([4,-2]))

# 三阶段测试
* 解析法求梯度的算法扩展到求Hesse阵
* 无约束优化：牛顿法

In [ ]:
from search import _dim
def _hesse(f, x):
    x_ = []
    for a in range(10):
        x_.append(symbols('x'+str(a),real=True))  #设置符号变量
    f_ = f(x_)   # f的正则表达式f_
    dimensions = _dim(f_)
    
    item={}
    for i in range(dimensions):
        item[x_[i]] = x[i]

    dx = []
    grad = []
    for a in range(dimensions):
        dx.append(diff(f_, symbols('x'+str(a),real=True))) #求偏导
        grad.append(dx[a].subs(item)) #求梯度 
    #print(f_)
    print(type(grad[0]))

    hesse = []
    for i in range(dimensions):
        ddx_i = []
        hesse_i = []
        for j in range(dimensions):
            ddx_i.append(diff(dx[i], symbols('x'+str(j),real=True)))
            #print(dx[i],'e',ddx_i[j])
            hesse_i.append(ddx_i[j].subs(item))
        hesse.append(hesse_i)
    print(hesse)
    return hesse

In [ ]:
def fun(x):
    return 100*(x[1]-x[0]**2)**2+(1-x[0])**2 #+sin(x[1])

In [ ]:
from search import _grad
#_grad(fun, [1,2])
h =  _hesse(fun, [1,2])
h = [list(map(float, e)) for e in h]
#list(map(float,h[0]))
np.linalg.inv(h)

In [ ]:
np.linalg.inv([[402, -400], [-400, 200]])

In [ ]:
np.linalg.norm([1], np.inf)

# 四阶段测试
* 正式测试：牛顿法+非精确一维搜索 无约束最优化


In [ ]:
from newton import opt
from sympy import *

def fun(x):
    return 100*((x[1]-x[0]**2)**2+(x[2]-x[1]**2)**2)+((1-x[0])**2)+sin(x[0])#+(1-x[1])**2)
def fun2(x):
    return 100*((x[1]-x[0]**2)**2+(x[2]-x[1]**2)**2)+((1-x[0])**2+(1-x[1])**2)

In [ ]:
opter = opt(0.3,0.6)
opter.add_fun(fun2)

x=[0,0,0]
x_min = opter.opt_newtown(x)

print(x_min)
print(fun2(x_min)-fun2(x))

In [ ]:
opter = opt(0.2,0.9)
opter.add_fun(fun)

x = [0,0,0]
x_min = opter.opt_newtown(x)
print(x_min)
print(fun(x_min)-fun(x))
print(float(fun(x_min)-fun([1,1,1])))

In [ ]:
opter = opt(0.3,0.6)
opter.add_fun(fun)

x=[0,0,0]
x_min=opter.opt_newtown(x)
print(x_min)
print(fun(x_min)-fun(x))

## 额外测试（捞一下同学）
* 拟牛顿方法

In [3]:
from newton import opt
from sympy import *

def fun(x):
    return 100*((x[1]-x[0]**2)**2)+((1-x[0])**2+(1-x[1])**2) #+ sin(x[0])

opter = opt(0.2,0.6)
opter.add_fun(fun)

x0 = [2.1,0.3]
x_min = opter.opt_dfp(x0)
print(x_min)

触发停机条件：梯度小于阈值，运行正常
[1.00000000001488 1.00000000002957]


很遗憾，目前还不能使用，拟牛顿算法似乎不太稳定

In [9]:
from search import _grad

_grad(fun, [0.892326896222099,0.792227096222099])

[1.21958450740902, -1.21958450740894]

In [2]:
import numpy as np
y = np.array([1,3])
print(np.dot(y.T, y))
print(np.dot(y, y.T))
print(y.T@y)
print(y@y.T)

10
10
10
10


坏了，好像是dot的问题
看来要把原函数中的一维数组reshape一下

In [ ]:
def opt_dfp(f, x0, max_iter=1000, delta=1e-10, rho=0.3, sigma=0.6):
    ''' 用拟牛顿法+非精确一维搜索完成无约束最优化 '''
    searcher = Line_Searcher(rho, sigma)
    searcher.add_fun(f)

    x = x0
    d = np.array([-float(e) for e in _grad(f, x)])
    H = np.eye(len(x)) # H0 = I
    stop = 0  # 停机条件 0:达到最大迭代次数

    for iter in range(max_iter):
        alpha = searcher.WolfePowell(x, d, 1.)
        x_new = x + alpha * d
            
        if np.linalg.norm(_grad(f, x_new), np.inf) < delta: #np.linalg.norm(np.array([float(e) for e in x_new-x])) < delta:
            '''修改停机条件'''
            stop = 1 # 停机条件 1:梯度阻尼
            break
        s = (x_new - x).reshape(len(x),1)
        y = (_grad(f, x_new) - np.array([float(e) for e in _grad(f, x)])).reshape(len(x),1)
        #print(np.dot(y, y.T))
        H = H - np.dot(np.dot(H, np.dot(y, y.T)), H) / np.dot(np.dot(y.T, H), y) + np.dot(s, s.T) / np.dot(s.T, y)
        d = -np.dot(H, _grad(f, x_new)) # 搜索方向
        x = x_new
        
    if stop == 0:
        print("触发停机条件：达到最大迭代次数，注意此时可能未找到最优解，", end='')
        print("请增加迭代次数或增大梯度阈值重试。")
    elif stop == 1:
       print("触发停机条件：梯度小于阈值，运行正常")        
        
    return x